In [ ]:
!git clone https://github.com/nkt780426/Bachelor-s-Project.git

In [ ]:
!mv Bachelor-s-Project/going_modular .

In [1]:
import os

import torch
from torchinfo import summary
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torchvision import transforms
from torchvision.transforms import InterpolationMode

from going_modular.dataloader.magface import create_magface_dataloader
from going_modular.model.MagFaceRecognition import MagFaceRecognition
from going_modular.train_eval.train import fit
from going_modular.loss.MagLoss import MagLoss
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.learningrate import NormalMapLR
import warnings
warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'normalmap',
    'train_dir': './Dataset/Normal_Map/train',
    'test_dir': './Dataset/Normal_Map/test',
    
    # Cấu hình train
    'backbone': 'iresnet18',
    'epochs': 1000,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 224,
    'num_class': len(os.listdir('./Dataset/Normal_Map/train')),
    'embedding_size': 512,
    
    'learning_rate': 2,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # Hàm m(ai) giúp thay đổi ai từ 0.25 đến 1.6
    'scale': 64,
    'lambda_g': 20,
    'l_margin': 0.45, 
    'u_margin': 0.8,
    'l_a': 10, 
    'u_a': 110,
}

train_transform = transforms.Compose([
    transforms.RandomCrop(224),
])

test_transform = transforms.Compose([
    transforms.Resize((224,224), interpolation=InterpolationMode.NEAREST)
])

train_dataloader, val_dataloader = create_magface_dataloader(CONFIGURATION, train_transform, test_transform)

model = MagFaceRecognition(CONFIGURATION).to(device)
criterion = MagLoss(conf = CONFIGURATION)

optimizer = torch.optim.Adam(model.parameters(), lr=CONFIGURATION['learning_rate'])
scheduler = NormalMapLR(optimizer=optimizer, fixed_epoch=2, fixed_lr=20, start_warm_lr=2)
checkpoint_path = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/' + CONFIGURATION['backbone'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1)
earlystop_dir = os.path.abspath('checkpoint/magface/' + CONFIGURATION['type'] + '/' + CONFIGURATION['backbone'] + '/models')
early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['val_euclidean_accuracy', 'val_cosine_accuracy', 'val_auc_euclidean', 'val_auc_cosine'],
    patience=40,
    mode='max',
    verbose=1,
    save_dir=earlystop_dir,
    start_from_epoch=40
)

2024-12-15 13:51:36.651087: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 13:51:37.555847: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
fit(CONFIGURATION, 0, model, device, train_dataloader, val_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 1:
	train: loss 3.766 | loss id   3.74 | top_1_acc 0.0060 | top_5_acc 0.0217 | acc_eu: 0.618 | acc_cos: 0.637 | auc_eu: 0.579 | auc_cos: 0.588
	val: acc_eu: 0.693 | acc_cos: 0.728 | auc_eu: 0.583 | auc_cos: 0.582
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/checkpoint.pth
Epoch 2:
	train: loss 2.170 | loss id   2.15 | top_1_acc 0.0052 | top_5_acc 0.0329 | acc_eu: 0.332 | acc_cos: 0.397 | auc_eu: 0.558 | auc_cos: 0.599
	val: acc_eu: 0.456 | acc_cos: 0.597 | auc_eu: 0.626 | auc_cos: 0.696
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/checkpoint.pth
Epoch 3:
	train: loss 0.621 | loss id   0.60 | top_1_acc 0.0157 | top_5_acc 0.0433 | acc_eu: 0.598 | acc_cos: 0.598 | auc_eu: 0.557 | auc_cos: 0.562
	val: acc_eu: 0.739 | acc_cos: 0.397 | auc_eu: 0.636 | auc_cos: 0.678
	Savi

In [2]:
checkpoint = torch.load('checkpoint/magface/' + CONFIGURATION['type'] + '/' + CONFIGURATION['backbone'] + '/models/checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
print(epoch)

172


In [3]:
fit(CONFIGURATION, 168, model, device, train_dataloader, val_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 169:
	train: loss 0.952 | loss id   0.93 | top_1_acc 0.6024 | top_5_acc 0.8871 | acc_eu: 0.839 | acc_cos: 0.820 | auc_eu: 0.863 | auc_cos: 0.896
	val: acc_eu: 0.861 | acc_cos: 0.869 | auc_eu: 0.789 | auc_cos: 0.815
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/checkpoint.pth
	Saved best model weights for 'val_euclidean_accuracy' at epoch 169 to '/media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/best_val_euclidean_accuracy.pth'
	Saved best model weights for 'val_cosine_accuracy' at epoch 169 to '/media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/best_val_cosine_accuracy.pth'
	Saved best model weights for 'val_auc_euclidean' at epoch 169 to '/media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresn

KeyboardInterrupt: 

In [2]:
checkpoint = torch.load('checkpoint/magface/' + CONFIGURATION['type'] + '/' + CONFIGURATION['backbone'] + '/models/checkpoint.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
fit(CONFIGURATION, 172, model, device, train_dataloader, val_dataloader, criterion, optimizer, scheduler, early_stopping, modle_checkpoint)

Epoch 173:
	train: loss 1.127 | loss id   1.10 | top_1_acc 0.6794 | top_5_acc 0.9185 | acc_eu: 0.876 | acc_cos: 0.853 | auc_eu: 0.888 | auc_cos: 0.924
	val: acc_eu: 0.858 | acc_cos: 0.849 | auc_eu: 0.805 | auc_cos: 0.759
	Saving model and optimizer state to /media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/checkpoint.pth
	Saved best model weights for 'val_euclidean_accuracy' at epoch 173 to '/media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/best_val_euclidean_accuracy_173.pth'
	Saved best model weights for 'val_cosine_accuracy' at epoch 173 to '/media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalmap/iresnet18/models/best_val_cosine_accuracy_173.pth'
	Saved best model weights for 'val_auc_euclidean' at epoch 173 to '/media/vohoang/WorkSpace/ubuntu/projects/in-process/Bachelor-s-Project/checkpoint/magface/normalm

KeyboardInterrupt: 